# Galaxy PCA
This notebook will take you through the steps of performing PCA on the galaxy dataset

### Imports 

In [ ]:
import tensorflow as tf

tfk = tf.keras
tfkl = tf.keras.layers
import numpy as np
import matplotlib.pyplot as plt
import os, glob
import tqdm
import re

PATH_TO_DATA = "data/final_28x28_proc"

### Data ingest
This cell defines a function to load in the JPG-encoded images into a form we can easily use. This cell yields a set of images, shape (width x height x channels) which we concatenate along a new axis to get a stack of images, shape (image x width x height x channels)

In [ ]:
def load_image_from_path(path):
    """
    Decode a jpg image from the string filepath

    :param path: Path to image
    :return: 3D image tensor
    """
    raw_img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(raw_img, channels=3)
    img = tf.image.convert_image_dtype(img, 'float32') # pre-normalises to 0,1
    
    return img.numpy()

filelist = glob.glob(os.path.join(PATH_TO_DATA, "*", "*.jpg"))
all_data = np.array([load_image_from_path(f) for f in tqdm.tqdm(filelist, desc='Processing data')])

### Pre-processing
In this cell, we need to convert the images to greyscale to make the upcoming analysis easier. `all_data` needs to now have shape (images x width x height)
*Note:* this code will fail until you fix it!

In [ ]:
monochrome_data = "..." # write code here - either to take the mean, or pick out an individual component

fig, axes = plt.subplots(1, 5)

randidx = np.random.randint(0, len(monochrome_data), size=5)

for ax, example in zip(axes, randidx):
    ax.imshow(monochrome_data[example], cmap='Greys')
    ax.axis("off")

### PCA
In these next cells, we'll perform PCA on the dataset we've now made monochrome. Due to how PCA works, we need to flatten the images into a 2D vector, effectively converting them from n_images x 28 x 28 -> n_images x 784. We'll undo this afterwards!

In [ ]:
from sklearn.decomposition import PCA

# For sklearn we need to flatten all non-leading dimensions - don't worry, this isn't permanent!
pca_data = monochrome_data.reshape(len(monochrome_data), -1)
print("PCA input has shape {}".format(pca_data.shape))

### Actually doing the PCA
In this cell, we apply PCA to the data using the `PCA` routine from `scikit-learn`. We truncate at the first 20 principal components initially, but you can change this yourself to access the higher-order components.

For the `eigengalaxies` variable, we undo our initial flattening, reshaping the vectors to the original 28 x 28 image format. The first plot shows how the eigengalaxies look, and the second plot shows the fraction of explained variance (i.e. how good each component is at capturing the data)

In [ ]:
# N most prominent components, for example
pca_model = PCA(n_components=20).fit(pca_data)

eigengalaxies = pca_model.components_.reshape(pca_model.n_components, 28, 28)

fig, axes = plt.subplots(3, pca_model.n_components // 3)

for idx, (ax, eigengalaxy) in enumerate(zip(axes.ravel(), eigengalaxies)):
    ax.imshow(eigengalaxy, cmap='Greys')
    ax.axis("off")
    ax.set_title(f"PC {idx}")
    
plt.show()

plt.plot(1 - pca_model.explained_variance_ratio_)
plt.xlabel("Principal component")
plt.ylabel("Fraction of explained variance")

### How do galaxies look with truncated principal components?
* Change the value of `n_components` in the cell below, and observe how the reconstruction behaves.

* The line `truncated_imgs = np.einsum("ij,jkl->ikl", pca_coeffs, eigengalaxies)` does a lot in one line, and uses [Einstein summation](https://en.wikipedia.org/wiki/Einstein_notation) (for those familiar) to compactly write the below steps
   * `pca_coeffs` has shape (`n_images`, `n_components`)
   * `eigengalaxies` has shape (`n_components`, `width`, `height`)
   * For each of `n_images`, we do sum(coefficient_n * component_n) -> i.e. principal component 1 * eigengalaxy 1 + principal component 2 * eigengalaxy 2, building up our final image(s) one principal component at a time
   * This is effectively the `dot product` between the eigenvalues and eigenvectors, but because of the way the dimensions are oriented it's a little more work.

In [ ]:
pca_result = PCA(n_components=784).fit(pca_data)
pca_coeffs = pca_result.transform(pca_data)
eigengalaxies = pca_result.components_.reshape(pca_result.n_components, 28, 28)

print(pca_coeffs.shape)
print(eigengalaxies.shape)

randidx = np.random.randint(0, len(monochrome_data), size=5)

truncated_imgs = np.einsum("ij,jkl->ikl", pca_coeffs, eigengalaxies)

fig, axes = plt.subplots(3, 5)

fig.suptitle(f"PC <= {pca_result.n_components}")

for idx, example in zip(range(5), randidx):
    axes[0][idx].imshow(monochrome_data[example], cmap='Greys')
    axes[1][idx].imshow(truncated_imgs[example], cmap='Greys')
    axes[2][idx].imshow(monochrome_data[example] - truncated_imgs[example], cmap='Greys', vmin=-1, vmax=1)
    axes[0][idx].axis("off")    
    axes[1][idx].axis("off")
    axes[2][idx].axis("off")

axes[0][0].set_ylabel("Original")
axes[1][0].set_ylabel("Truncated")
axes[2][0].set_ylabel("Residual")
    
plt.show()

### How does the PCA latent space look?
In the below cells, we extract the morphology string from the file name - the `amend_morphology_str` function matches on filenames, and returns a normal set of strings

In [ ]:
morphology = np.array([f.split("/")[-1][:-4].split("_")[-1] for f in filelist])

def amend_morphology_str(morphstr):
    # Merge all ellipticals into one class
    if re.search("E.", morphstr):
        return "E"
    
    # Merge all barred spirals into one class
    if re.search("SB.", morphstr):
        return "SB"
    
    # Move transitional spirals into Irregular
    if re.search("Sm", morphstr):
        return "Irr"
    
    # All spirals in the same class
    if re.search("S.", morphstr):
        return "S"
    
    else:
        return morphstr
    
morphology_corr = np.array([amend_morphology_str(s) for s in morphology])

for m in np.unique(morphology_corr):
    print("{}: {} of {}".format(m, (morphology_corr == m).sum(), len(morphology_corr)))
    
fig, ax = plt.subplots(1, len(np.unique(morphology_corr)))

print("Example morphology classes")
for i, m in enumerate(np.unique(morphology_corr)):
    idx = np.random.choice(np.argwhere(morphology_corr == m).flatten())
    ax[i].imshow(all_data[idx])
    ax[i].set_title(m)
    ax[i].axis("off")

In [ ]:
pca_coeffs = PCA(n_components=2).fit_transform(pca_data)

for group in ['E', 'Irr', 'S', 'SB', 'NA']:
    subset = morphology_corr == group
    
    plt.scatter(*pca_coeffs[subset].T, s=1, label=group)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

### Does PCA divide the classes well?
* The plot below shows PC1 and PC2, and colours the points by their label.
* Try changing the groups present in the list below, and see which classes can be distinguished from each other. Does it make sense?

In [ ]:
pca_coeffs = PCA(n_components=2).fit_transform(pca_data)

for group in ['E', 'S']:
    subset = morphology_corr == group
    
    plt.scatter(*pca_coeffs[subset].T, s=1, label=group)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()